In [52]:
from transformers import (
    AutoConfig, BertConfig, BertTokenizer, 
    BertForSequenceClassification)
    
import sys   
import torch
import numpy as np
from transformers import BertForRetrieval
import csv
import sklearn.metrics
from collections import defaultdict

In [75]:
ckpt_dir = "/home/transformers-public/output_dirs/rappi_instacart_data_128_10_bs_64/checkpoint-100000"

In [76]:
config = AutoConfig.from_pretrained(ckpt_dir)
tokenizer = BertTokenizer.from_pretrained(
    ckpt_dir,
    do_lower_case=True,
)

model = BertForRetrieval.from_pretrained(
    ckpt_dir,
    from_tf=False,
    #config=config,
    #cache_dir='/home/sarahwooders_gmail_com/transformers/checkpoints'
)

In [77]:
use_gpu = False
if use_gpu:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

cpu


In [78]:
model = model.to(device)

### All labels

In [1]:
import pickle

rappi_clean = pickle.loads(open('rappi_clean_categories.pkl', 'rb').read())
labels = []
for category in rappi_clean: labels += category
labels = list(sorted(set(labels)))
print(len(labels))
print(labels[:5])

559
['accessories', 'accessories babies and children', 'accessories for babies', 'accessories men', 'alcohol']


In [80]:
msl_cat = 10
label_tokenization = tokenizer.batch_encode_plus(labels,
                                                 max_length=msl_cat,
                                                 pad_to_max_length=True)
label_tokenization = {k: torch.tensor(v).to(device) for k, v in label_tokenization.items()}
model_out = model(
        input_ids_a=label_tokenization['input_ids'], # We're going to ignore these
        attention_mask_a=label_tokenization['attention_mask'],
        token_type_ids_a=label_tokenization['token_type_ids'],
        input_ids_b=label_tokenization['input_ids'], 
        attention_mask_b=label_tokenization['attention_mask'],
        token_type_ids_b=label_tokenization['token_type_ids']
    )
label_embedding = model_out[2].detach().cpu().numpy()print(label_embedding.shape)

(182, 768)


In [8]:
category_level = 'category_4' # category_[1-4] where category_1 is shallowest 
all_samples = []
all_true_labels = []

#validation_data = pickle.loads(open('other_validation.pkl', 'rb').read())
validation_data = pickle.loads(open('instacart_validation.pkl', 'rb').read())

for row in validation_data:
    if category_level not in row: # not all rows have level 4 
        continue
    text = row['translated_title'] 
    desc = row['translated_description']
    label = row[category_level] 
    if label not in labels:
        print(label)
        raise ValueError("Bad label")
    all_samples.append((text, desc))
    all_true_labels.append(label)

In [9]:
len(all_samples)

477

In [4]:
all_samples[:5]

[('Tram.Inox Game My Best Friend With 3Pcs -23799/491',
  'Tram.Inox Game My Best Friend With 3Pcs -23799/491'),
 ('Chicco Tetina Tetina N.Feeling 6M Fl',
  'Chicco Nipple Teat N. Feeling 6M Fl - Prices including VAT'),
 ('Nail Polish Risque Love Mooring Love 8 mL',
  'Nail Polish Risque Love Mooring Love 8 mL'),
 ('50% in 2 Units Stain Remover Vanish 3L Bleach without',
  'Multi-purpose Vanish has 3x more benefits than chlorine, and can be used in both colored and white clothing, preserving fabrics.'),
 ('Gillette Mach3 Turbo 2 Shaver Charger',
  'Gillette Mach3 Turbo 2 Shaver Charger')]

In [84]:
all_title_embeddings = []
all_true_label_emebddings = []
all_pred_ranks = []
chunk = 5
for i in range(0, len(all_samples), chunk):
    samples = all_samples[i:i+chunk]
    true_labels = all_true_labels[i:i+chunk]
    msl_title_desc = 128
    title_tokenization = tokenizer.batch_encode_plus(samples,
                                                     max_length=msl_title_desc,
                                                     pad_to_max_length=True)
    title_tokenization = {k: torch.tensor(v).to(device) for k, v in title_tokenization.items()}
    true_label_tokenization = tokenizer.batch_encode_plus(true_labels,
                                                     max_length=msl_cat,
                                                     pad_to_max_length=True)
    true_label_tokenization = {k: torch.tensor(v).to(device) for k, v in true_label_tokenization.items()}
    model_out = model(
            input_ids_a=title_tokenization['input_ids'], # We're going to ignore these
            attention_mask_a=title_tokenization['attention_mask'],
            token_type_ids_a=title_tokenization['token_type_ids'],
            input_ids_b=true_label_tokenization['input_ids'], 
            attention_mask_b=true_label_tokenization['attention_mask'],
            token_type_ids_b=true_label_tokenization['token_type_ids']
        )
    titles_embedding = model_out[1].detach().cpu().numpy()
    # print(titles_embedding.shape)
    true_label_embedding = model_out[2].detach().cpu().numpy()
    # print(true_label_embedding.shape)
    all_title_embeddings.append(titles_embedding)
    all_true_label_emebddings.append(true_label_embedding)
    similarity = sklearn.metrics.pairwise.cosine_similarity(titles_embedding,
                                                           label_embedding
                                                          )
    rank_label_num = np.argsort(-similarity, axis=-1)
    for j in range(len(samples)):
        print(samples[j], true_labels[j])
        pred_label_nums = rank_label_num[j]
        for pln in pred_label_nums[:10]:
            print(pln, labels[pln])
        true_idx = labels.index(true_labels[j])
        pred_rank = list(pred_label_nums).index(true_idx)
        print("Predicted rank:", pred_rank)
        print("====")
        all_pred_ranks.append(pred_rank)

('Japanese Mafer Premium Roasted Peanuts with Lemon 180 g', 'Japanese Mafer Premium Roasted Peanuts with Lemon 180 g - Prices including VAT') snacks and confectionery
76 fresh pasta and pasta
36 clothing
143 pub
141 prime rappi
37 clothing care
147 salads
69 flour and mixtures to prepare
168 unisex clothing
1 accessories men
92 jams and spreads
Predicted rank: 34
====
('Staedtler Mars Plastic Rubber Poster 3 Units', '• Considered the best rubber in the world • Sliding sleeve for better handling • Maximum efficiency: formula that leaves little residue, facilitating cleaning.') office supplies
81 gardening
34 cleaning products
56 disposables
23 car wash
85 hardware store
79 furniture
12 bathrooms
38 coal, wood and spark
108 medical equipment and supplies
160 storage devices
Predicted rank: 13
====
('Organic Red Pitaya 500 g', 'Organic Red Pitaya 500 g') fruits and vegetables
114 natural products
168 unisex clothing
134 pet food
61 electro beauty
72 formulas and foods
90 incontinence prot

('Polypropylene Reading Lantern N Daiso 1 Und', 'Polypropylene Reading Lantern N Daiso 1 Und') lighting and electrical
120 organizers
59 e-books
14 beauty accessories
30 children accessories
1 accessories men
142 print
166 tires and wheels
117 office supplies
93 journals
36 clothing
Predicted rank: 59
====
('Small Clean Sponge Vegetable Cloth', 'Pano Vegetable Sponge Limppano Small - 1 UN') bathrooms
12 bathrooms
38 coal, wood and spark
81 gardening
79 furniture
87 home decor
97 kitchen
34 cleaning products
129 party decorations
167 unisex accessories
56 disposables
Predicted rank: 0
====
('Vcy Dermablend Base Fluida Facial 55', 'Vichy Dermablend Facial Fluid Base SPF35 Nº55 Active Principle: None MS Record: - Vichy Dermablend Fluid Base Color 55 SPF 35 high coverage without mask effect. Matifying effect. Hydration 24 hours. Ideal to uniform texture, correct dark circles and irregular tones. Ideal for all skin types. Weight: 52 gr - Dosage: - Manufactures: None - Brand: Vichy (Key Word

('Dragon Ball Super Vol. 3', 'Dragon Ball Super Vol. 3') paper books
95 keyboards
62 electronic supplies
117 office supplies
159 sports articles
161 strings
100 lighting and electrical
166 tires and wheels
44 consoles
13 battery
167 unisex accessories
Predicted rank: 16
====
('Iana Amazon Ethyl Alcohol 70% 100 mL', 'Iana Amazon Ethyl Alcohol 70% 100 mL') and first aid kit
176 wines
16 beers and ciders
94 juices and nectars
139 powdered soft drinks and syrup
61 electro beauty
42 cokteles alcohol
101 liqueurs and aperitifs
45 coolers
138 please rappi
38 coal, wood and spark
Predicted rank: 110
====
('Karpex Carpet Cleaner 500 mL', 'Karpex Carpet Cleaner 500 mL') cleaning
34 cleaning products
56 disposables
35 cleaning soaps and
23 car wash
81 gardening
33 cleaning
55 dishwashing detergent
103 maintenance
25 cars and highchairs
77 fresheners and candles
Predicted rank: 5
====
('Freedom - Philosophies: the Pleasure of Thinking', 'In this volume, Alexandre Carrasco traces an instigating iti

('Edamame', 'Portion of Green Soy Beans cooked and seasoned only with \\ Flor de Sal \\ for appetizer. Accompanying suggestion for alcoholic beverages or as a complement to a meal.') fruits and vegetables
146 rice grains and pasta
69 flour and mixtures to prepare
102 liquids, additives and oils
118 oils and vinegars
57 dj consoles
76 fresh pasta and pasta
58 doorstep cash
47 cream and derivatives
28 cheese
140 preserves
Predicted rank: 16
====
('Vigor Yogurt Every Day Coco 540 g', 'Vigor Yogurt Every Day Coco 540 g') yogurt and dairy drinks
180 yogurt and dairy drinks
112 milk
60 eggs
162 subscription
32 cigarettes
72 formulas and foods
26 cereal drinks
48 cycling
143 pub
94 juices and nectars
Predicted rank: 0
====
('Colored petal napkins 350 pcs', 'Colored petal napkins 350 pcs') pet accessories
43 computers
62 electronic supplies
127 paper books
166 tires and wheels
64 exercise machines
59 e-books
171 vehicle accessories
117 office supplies
24 cards
85 hardware store
Predicted rank:

('Che Guevara - Biographies - Pocket', '&quot;Let us be realistic, let us do the impossible.&quot; Ernesto Che Guevara (1928-1967) is an icon. Images of his face that are both severe and hopeful (based on the famous photograph taken by Cuban Alberto Korda) are widespread in the world, even ingrained in pop culture. But what is hidden behind this portrait, behind the symbol of rebellion and hope? This book proposes to answer that question, following the trajectory of the young Argentine medical student on an initiatic journey through Latin America and the man who joins a revolutionary movement, overthrows the Cuban dictator Fulgencio Batista with Fidel Castro and ends up promote a revolutionary court. Creator of the “work and re-education” camps, Che Guevara held several important positions in the Cuban government. After disappearing from national political life and fighting in Congo, the man who is credited with the phrase &quot;it must be hardened, but without losing tenderness&quot; 

...

('Pastrami Peru Bern Sliced Condiment', 'Pastrami Peru Bern Sliced Condiment') delicatessen
76 fresh pasta and pasta
28 cheese
140 preserves
148 sandwiches and empanadas
168 unisex clothing
75 fresh flowers and plants
130 percussion
107 meats
152 seasonings
78 fruits and vegetables
Predicted rank: 50
====
('Dove Nutrition Oil Conditioner 400 mL', 'Dove Nutrition Oil Conditioner 400 mL') hair care
106 massages
158 sports
178 woman's accesories
90 incontinence protection
83 hair care
51 dermatological care
159 sports articles
63 energy and isotonic
17 breastfeeding and accessories
177 woman cloth
Predicted rank: 4
====
('Koli D3 1,000Ui 60 Coated Tablets', 'Koli D3 1,000Ui 60 Coated Tablets') vitamins and minerals
69 flour and mixtures to prepare
76 fresh pasta and pasta
75 fresh flowers and plants
78 fruits and vegetables
37 clothing care
1 accessories men
36 clothing
168 unisex clothing
118 oils and vinegars
89 ice cream and desserts
Predicted rank: 28
====
('Cream 6 Pork', 'La Sopera 

('Silicone Buba 7764 Textured Teether', 'Silicone Buba 7764 Textured Teether') baby accessories
1 accessories men
57 dj consoles
52 details and accessories
168 unisex clothing
161 strings
30 children accessories
130 percussion
95 keyboards
85 hardware store
0 accessories for babies
Predicted rank: 38
====
('Leo Japanese Peanut 180 Gr', 'Leo Japanese Peanut 180 Gr - Prices including VAT') snacks and confectionery
141 prime rappi
36 clothing
76 fresh pasta and pasta
40 coffee, tea and milk modifiers
143 pub
41 coffees and teas ready to drink
37 clothing care
115 notepads and notes
39 coffee, chocolates and teas
168 unisex clothing
Predicted rank: 47
====
('Sortdo Photographic Machine Teether', 'Sortdo Photographic Machine Teether') baby accessories
7 baby accessories
0 accessories for babies
30 children accessories
8 baby clothes
167 unisex accessories
68 fitness
36 clothing
1 accessories men
133 pet accessories
161 strings
Predicted rank: 0
====
('Phebo Soap Lemon Sicilian', 'Phebo Soap

('Cafe Capsule Torra Media Illy 130g', 'Cafe Capsule Torra Media Illy 130g') coffee, tea and milk modifiers
167 unisex accessories
172 vitamins and minerals
116 nutritional health
132 pest control
153 sexual health
145 repair cell
169 unisex shoes
57 dj consoles
17 breastfeeding and accessories
54 digestive health
Predicted rank: 78
====
('Calabrian Empanada With Catupiry (440 Gr)', 'Empanada Catupiry Calabresa 440 g') sandwiches and empanadas
130 percussion
141 prime rappi
168 unisex clothing
148 sandwiches and empanadas
57 dj consoles
95 keyboards
138 please rappi
44 consoles
147 salads
161 strings
Predicted rank: 3
====
('Café Italiano Compagnia Dell&#39;Arabica Cápsulas', 'Café Italiano Compagnia Dell&#39;Arabica Cápsulas - 52 G') coffee, tea and milk modifiers
64 exercise machines
168 unisex clothing
167 unisex accessories
115 notepads and notes
6 audiobooks
57 dj consoles
1 accessories men
120 organizers
75 fresh flowers and plants
36 clothing
Predicted rank: 30
====
('Clips 2/0 

('Milk Lacta Chocolate 135 g', 'Buy Milk Chocolate Lacta at Drugstore Araujo. Delicious Milk Lacta Chocolate, Now With More Flavor For You To Share This Joy. Enjoy Our Offers!') snacks and confectionery
112 milk
180 yogurt and dairy drinks
48 cycling
60 eggs
94 juices and nectars
116 nutritional health
152 seasonings
47 cream and derivatives
163 sugar and sweeteners
28 cheese
Predicted rank: 107
====
('Deep Keratinization Silk Mask 1 Kg', 'Deep Keratinization Silk Mask 1 Kg') hair care
174 waxing and shaving
168 unisex clothing
75 fresh flowers and plants
37 clothing care
28 cheese
36 clothing
159 sports articles
120 organizers
141 prime rappi
158 sports
Predicted rank: 94
====
('Neugebauer Chocolate Peanut 95 g', 'Neugebauer Chocolate Peanut 95 g') snacks and confectionery
80 games
89 ice cream and desserts
163 sugar and sweeteners
72 formulas and foods
96 kids' shoes
24 cards
59 e-books
48 cycling
99 lessons
127 paper books
Predicted rank: 47
====
('Natural Company Quinoa and Kale Ri

('Lc Green Tea 50 g', 'Lc Green Tea 50 g') coffee, chocolates and teas
41 coffees and teas ready to drink
40 coffee, tea and milk modifiers
39 coffee, chocolates and teas
61 electro beauty
176 wines
93 journals
143 pub
94 juices and nectars
162 subscription
60 eggs
Predicted rank: 2
====
('Diet Milk Chocolates Kopenhagen 105 g', 'Diet Milk Chocolates Kopenhagen 105 g') yogurt and dairy drinks
143 pub
89 ice cream and desserts
36 clothing
11 bakery
39 coffee, chocolates and teas
48 cycling
162 subscription
1 accessories men
59 e-books
124 otrqs prepared foods
Predicted rank: 84
====
('Washing Powder Maxx Urca 3Kg', 'Washing Powder Maxx Urca 3Kg') clothing care
158 sports
159 sports articles
14 beauty accessories
95 keyboards
155 skin care
46 costumes
6 audiobooks
117 office supplies
5 audio & sound
120 organizers
Predicted rank: 49
====
('May Cavado Wild Animals Animal Print Coconut Water', 'May Cavado Wild Animals Animal Print Coconut Water') waters
168 unisex clothing
167 unisex acces

In [89]:
print(np.mean(all_pred_ranks))

41.17066666666667


In [90]:
rank_by_category = defaultdict(list)

In [91]:
for s, t, p in zip(all_samples, all_true_labels, all_pred_ranks):
    rank_by_category[t].append(p)
    print(s, t, p)

('Japanese Mafer Premium Roasted Peanuts with Lemon 180 g', 'Japanese Mafer Premium Roasted Peanuts with Lemon 180 g - Prices including VAT') snacks and confectionery 34
('Staedtler Mars Plastic Rubber Poster 3 Units', '• Considered the best rubber in the world • Sliding sleeve for better handling • Maximum efficiency: formula that leaves little residue, facilitating cleaning.') office supplies 13
('Organic Red Pitaya 500 g', 'Organic Red Pitaya 500 g') fruits and vegetables 97
('Rappicombo Powerade Tropical Fruits + cereal bars', 'Rappicombo includes Powerade Tropical Fruits 500ml PLU: 903981 and Higher Rotation Cereal Bars PLU: 94568') snacks and confectionery 110
('Tram.Inox Game My Best Friend With 3Pcs -23799/491', 'Tram.Inox Game My Best Friend With 3Pcs -23799/491') accessories for babies 43
('Chicco Tetina Tetina N.Feeling 6M Fl', 'Chicco Nipple Teat N. Feeling 6M Fl - Prices including VAT') accessories for babies 11
('Mentos Pure White Mint 56 g', 'Mentos Pure White Mint 56 g.

('Glass Cleaner See Vidrex Cristal Spray', 'Glass Cleaner See Vidrex Cristal Spray') cleaning 101
('Pediasure Vanilla Milk 900 g', 'Pediasure Powder Provides Complete And Balanced Nutrition For Children 1 And 10 Years Old - Can Be Used As A Total Nutrition Support, Or As A Nutritional Supplement - Pediasure&#39;s Unique Formulation Containing Low Lactose Levels Makes You Go') milk 0
('Celluctiv Body slim Sanavita 300 g', 'Celluctiv Body slim Sanavita 300 g') nutritional health 3
('Masglo Nail Polish', 'Passion Masglo Enamel, 135 Milliliter (s)') makeup 126
('Ferm Elege Ame Dairy Drink 900 Gr', 'Ferm Elege Ame Dairy Drink 900 Gr') yogurt and dairy drinks 2
('Weihenstep Beer Kit. With 2 Bottle + 1Cop. 500 mL', 'Weihenstep Beer Kit. With 2 Bottle + 1Cop. 500 mL') beers and ciders 0
('White Asparagus CasaVerde', 'White Asparagus CasaVerde - 2112 G') preserves 49
('Essence pencils', 'Lardur Marp Eye Pencil - PLU: 1192774') school supplies 148
('Talc Memphis Lavender 100 g', 'Talc Memphis La

In [92]:
for cat, ranks in rank_by_category.items():
    print(cat, np.mean(ranks), len(ranks))

snacks and confectionery 48.608695652173914 115
office supplies 5.333333333333333 3
fruits and vegetables 40.827586206896555 29
accessories for babies 25.7 10
other drugs 48.925925925925924 27
deodorants 50.3125 16
makeup 58.125 32
cleaning products 64.33333333333333 3
cleaning 78.0625 16
bakery 28.944444444444443 18
electro beauty 23.5 2
seasonings 34.84375 32
games 52.0 10
hair care 53.96 50
coffees and teas ready to drink 21.571428571428573 7
pet accessories 20.25 8
accessories men 13.2 5
lighting and electrical 45.0 3
bathrooms 5.0 3
woman cloth 113.2 5
beauty accessories 18.9 10
fresheners and candles 30.285714285714285 7
hardware store 0.0 1
cheese 12.9 10
other frozen foods 98.6 5
ice cream and desserts 29.1 10
beers and ciders 26.75 16
skin care 30.642857142857142 28
hygiene and care 62.0 40
kitchen 39.86666666666667 15
oral care 25.0625 16
sandwiches and empanadas 46.5 2
paper books 25.266666666666666 30
and first aid kit 103.5 6
men clothes 58.166666666666664 6
dishwashing de